In [1]:
from os.path import join as pjoin
from os import listdir
from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import rankdata

from evaluate import get_nan_percent, get_performance, tolerant_mean

In [2]:
bookkeeping_path = ('session_info.csv')

mouse_codes = []
dates = []
versions = []
performances = []
session_indices = np.array([], dtype=int)

for mouse_code in listdir('data'):
    mice_path = pjoin('data', mouse_code)

    cur_mouse_code = []
    cur_date = []
    cur_version = []
    cur_performance = []

    for session in listdir(mice_path):
        date_version = '.'.join(session.split('.')[:-1])
        date = date_version.split('-')[0]
        version = date_version.split('-')[1]
        session_path = pjoin(mice_path, session)
        session_data = pd.read_csv(session_path)
        leftP = session_data['leftP']
        choices = session_data['choices']
        try:
            performance = get_performance(choices=choices, leftP=leftP, version=version)
        except RuntimeError:
            continue
        cur_mouse_code.append(mouse_code)
        cur_date.append(date)
        cur_version.append(version)
        cur_performance.append(performance)
        
    cur_session_indices = rankdata(cur_date).astype(int)
    mouse_codes += cur_mouse_code
    dates += cur_date
    performances += cur_performance
    versions += cur_version
    session_indices = np.concatenate((session_indices, cur_session_indices))

data = pd.DataFrame({'mouse_code': mouse_codes, 
                    'date': dates,
                    'performance': performances,
                    'version': versions,
                    'session_index': session_indices})

data.to_csv(bookkeeping_path, index=False)

['22.08.12' '22.08.02' '22.07.27' '22.08.16' '22.08.03' '22.08.23'
 '22.08.11' '22.08.08' '22.07.27' '22.07.18' '22.07.12' '22.07.13'
 '22.08.24' '22.08.15' '22.08.10' '22.08.19' '22.07.25' '22.07.05'
 '22.07.28' '22.07.06' '22.07.20' '22.08.15' '22.07.29' '22.08.15'
 '22.06.30' '22.08.09' '22.07.01' '22.08.22' '22.07.07' '22.07.04'
 '22.07.14' '22.08.18' '22.07.01' '22.08.01' '22.08.17']
[25 19 14 29 20 34 24 21 14 11  8  9 35 27 23 32 13  5 16  6 12 27 17 27
  1 22  2 33  7  4 10 31  2 18 30]
['22.08.12' '22.08.02' '22.04.04' '22.07.26' '22.06.07' '22.06.17'
 '22.07.27' '22.07.01' '22.03.30' '22.04.04' '22.04.14' '22.04.19'
 '22.04.22' '22.08.18' '22.08.24' '22.03.10' '22.03.28' '22.05.03'
 '22.04.12' '22.03.22' '22.03.17' '22.06.30' '22.05.02' '22.03.29'
 '22.08.03' '22.03.21' '22.03.16' '22.04.27' '22.06.29' '22.05.25'
 '22.08.23' '22.07.06' '22.04.05' '22.08.10' '22.03.17' '22.08.08'
 '22.03.25' '22.04.15' '22.08.01' '22.08.11' '22.05.04' '22.03.15'
 '22.06.02' '22.05.31' '22.04.2